# Costs

We cannot (yet) run the large ('fault-tolerant scale') quantum algorithms expressed in Qualtran. Instead, a major research goal is to estimate the resources required to implement interesting quantum algorithms. Resources can be time, qubits, T gates, precision, depth, or any number of things. Throughout Qualtran, we use the less ambiguous term "costs" to represent these quantities.

In the research literature, there is a surface level agreement on what costs to count: Often we count gates like T, Toffoli, Clifford, and Rotations; And the number of qubits is tabulated. These are the most probable limiting factors of implementing these algorithms on real quantum computers. When it comes to details, agreement on costs is not guaranteed. Researchers may want to consider additional target gatesets or architectures; disagree on how to lump or count cliffords of various sizes; and treat rotations in a variety of ways.

In Qualtran, we provide a set of configurable `CostKey`s that can be used to query algorithms expressed as bloqs. Developers can also implement their own costs by overriding `CostKey` to provide even more customization.

## Getting costs

The following functions can be used to query costs for a bloq. Each takes a bloq and a cost key:

 - `get_cost_value`
 - `get_cost_cache`

The former will return a single value whereas the latter will return a dictionary mapping every bloq for which a cost was computed to its value.

In [ ]:
from qualtran.resource_counting import get_cost_value, get_cost_cache

Before we start, we'll create an example bloq with enough complexity to be interesting:

In [ ]:
# Construct a PrepTHC bloq for demonstration purposes

from qualtran.bloqs.chemistry.thc.prepare_test import build_random_test_integrals

num_spat = 4
num_mu = 8
t_l, eta, zeta = build_random_test_integrals(num_mu, num_spat, seed=7)
prep_thc = PrepareTHC.from_hamiltonian_coeffs(t_l, eta, zeta, num_bits_state_prep=8)
prep_thc

`get_cost_value` takes a `Bloq` and a `CostKey` and returns a value. The type of value depends on the particular cost key. For example, if we ask for the qubit count by using the `QubitCount` cost key, we get an integer.

In [ ]:
from qualtran.resource_counting import get_cost_value, QubitCount

get_cost_value(prep_thc, QubitCount())

`get_cost_cache` has the same input arguments, but instead returns a dictionary containing the (sub-)costs for each bloq that was encountered during the recursive computation. In the following demo, we only print the top 5 entries to avoid overwhelming the output.

In [ ]:
from qualtran.resource_counting import get_cost_cache

cost_cache = get_cost_cache(prep_thc, QubitCount())
print(f'cost_cache contains {len(cost_cache)} entries. Displaying top five.\n')

top_five = sorted(cost_cache.items(), key=lambda x: x[1])[-5:]
for bloq, val in top_five:
    print(f'{bloq}: {val} qubits')

## Configurable Cost Keys

The behavior of a cost computation can be modified by arguments to the cost key. The `QubitCount()` cost is simple: it has no parameters and the value of the cost is a simple integer. `QECGatesCost` will similarly give counts of pre-selected gates. 

In [ ]:
from qualtran.resource_counting import QECGatesCost

get_cost_value(prep_thc, QECGatesCost())

Here, we can see that the output is a `GateCounts` data class that tallys up the number of T, Toffoli, CSwap (aka Fredkin), AND, and Clifford Gates. 

For more control, you can use the `BloqCounts` `CostKey` to list specific bloqs to count.

In [ ]:
from qualtran.resource_counting import BloqCount
from qualtran.bloqs.basic_gates import ZGate, SGate, TGate

z_rots = BloqCount(gateset_bloqs=[ZGate(), SGate(), TGate()], gateset_name='z rots')
get_cost_value(prep_thc, z_rots)

## Querying Multiple Costs

The `query_costs` function will get multiple costs for multiple bloqs simultaneously. Its output is suitable for annotating a call graph diagram

In [ ]:
from qualtran.resource_counting import query_costs

from qualtran.drawing import GraphvizCallGraph
from qualtran.resource_counting.generalizers import ignore_split_join, ignore_alloc_free

costs = query_costs(prep_thc, [QECGatesCost(), QubitCount()])
g, _ = prep_thc.call_graph(max_depth=2, generalizer=[ignore_split_join, ignore_alloc_free])
GraphvizCallGraph(g, costs).get_svg()

This code snippet forms the basis of the convenience function for displaying a pre-selected set of cost keys in a call graph format

In [ ]:
from qualtran.drawing import show_call_graph
show_call_graph(prep_thc, max_depth=2)

## Simple Example

In [ ]:
import sympy
from qualtran.bloqs.basic_gates import CSwap


n = sympy.Symbol('n', positive=True, integer=True)
cswap = CSwap(n)
show_call_graph(cswap)